In [4]:
import qelos as q
import random
import torch
import torch.nn as nn
import torch.backends.cudnn as cudnn
import torch.optim as optim
from torch.autograd import Variable
import os
import numpy
import sklearn
import sklearn.datasets

from matplotlib import pyplot
from matplotlib import gridspec
%matplotlib inline
import IPython
from datetime import datetime as dt
import pandas as pd

In [5]:
opt_onesided = False     # use two-sided or one-sided constraint

opt_penalty_weight = 10.0 # penalty weight term lambda

opt_lip_mode = "WGAN"
opt_dataset = 'swissroll' # 8gaussians | swissroll | 25gaussians
opt_niter = 5000
opt_batchSize=256
opt_lrD = 0.00005 # learning rate for Critic, default=0.00005
opt_lrG = 0.00005 # learning rate for Generator, default=0.00005
opt_beta1=0.5 # beta1 for adam. default=0.5
opt_cuda = torch.cuda.is_available
opt_clamp_lower = -0.01 #default -0.01
opt_clamp_upper =  0.01 #default  0.01
opt_Diters = 10 # number of D iters per each G iter
opt_adam = False  # Whether to use adam (default False is rmsprop)
opt_prefix = None # whether to write images (=prefix of type string) or show in notebook (=None)

opt_manualSeed = 15042017
print("Random Seed: ", opt_manualSeed)
random.seed(opt_manualSeed)
numpy.random.seed(opt_manualSeed)
torch.manual_seed(opt_manualSeed)
cudnn.benchmark = True

('Random Seed: ', 15042017)


In [6]:
# Dataset generator largely form Improved Training of Wasserstein GAN code (see link above)
def inf_train_gen(DATASET='8gaussians', BATCH_SIZE=opt_batchSize):
    numpy.random.seed(1234)
    if DATASET == '25gaussians':
        dataset = []
        for i in range(100000//25):
            for x in range(-2, 3):
                for y in range(-2, 3):
                    point = numpy.random.randn(2)*0.05
                    point[0] += 2*x
                    point[1] += 2*y
                    dataset.append(point)
        dataset = numpy.array(dataset, dtype='float32')
        numpy.random.shuffle(dataset)
        dataset /= 2.828 # stdev
        while True:
            for i in range(len(dataset)//BATCH_SIZE):
                yield torch.from_numpy(dataset[i*BATCH_SIZE:(i+1)*BATCH_SIZE])

    elif DATASET == 'swissroll':

        while True:
            data = sklearn.datasets.make_swiss_roll(
                n_samples=BATCH_SIZE, 
                noise=0.25
            )[0]
            data = data.astype('float32')[:, [0, 2]]
            data /= 7.5 # stdev plus a little
            yield torch.from_numpy(data)

    elif DATASET == '8gaussians':
    
        scale = 2.
        centers = [
            (1,0),
            (-1,0),
            (0,1),
            (0,-1),
            (1./numpy.sqrt(2), 1./numpy.sqrt(2)),
            (1./numpy.sqrt(2), -1./numpy.sqrt(2)),
            (-1./numpy.sqrt(2), 1./numpy.sqrt(2)),
            (-1./numpy.sqrt(2), -1./numpy.sqrt(2))
        ]
        centers = [(scale*x,scale*y) for x,y in centers]
        while True:
            dataset = []
            for i in range(BATCH_SIZE):
                point = numpy.random.randn(2)*.02
                center = random.choice(centers)
                point[0] += center[0]
                point[1] += center[1]
                dataset.append(point)
            dataset = numpy.array(dataset, dtype='float32')
            dataset /= 1.414 # stdev
            yield torch.from_numpy(dataset)

In [7]:
ds1 = [next(inf_train_gen("swissroll")) for i in range(100)]
ds2 = [next(inf_train_gen("swissroll")) for i in range(100)]
for ds1e, ds2e in zip(ds1, ds2):
    assert(numpy.allclose(ds1e.numpy(), ds2e.numpy()))

In [8]:
# Generates and saves a plot of the true distribution, the generator, and the
# critic.
# largely form Improved Training of Wasserstein GAN code (see link above)
class ImageGenerator:
  def __init__(self, netG, netD, prefix='frame', noise_dim=2):
    self.prefix = prefix
    self.frame_index = 1
    self.noise_dim = noise_dim
    self.netG = netG
    self.netD = netD
    
  def __call__(self, true_dist, perturbed, losses):
    try:
        N_POINTS = 128
        RANGE = 2

        points = numpy.zeros((N_POINTS, N_POINTS, 2), dtype='float32')
        points[:,:,0] = numpy.linspace(-RANGE, RANGE, N_POINTS)[:,None]
        points[:,:,1] = numpy.linspace(-RANGE, RANGE, N_POINTS)[None,:]
        points = points.reshape((-1,2))
        points = Variable(torch.from_numpy(points))
        if true_dist.is_cuda:
            points.cuda()

        noise = torch.FloatTensor().resize_(opt_batchSize, 2)
        noise.normal_(0, 1)
        noise = Variable(noise)
        if true_dist.is_cuda:
            noise.cuda()

        fake = self.netG(noise)
        samples = fake.data.cpu().numpy()

        disc_points = self.netD(points)

        disc_map = disc_points.data.cpu().numpy()
        disc_map = (disc_map - numpy.min(disc_map)) / numpy.max(disc_map) * 8
        disc_map = numpy.log(disc_map + 0.25)

        pyplot.figure(num=1, figsize=(10,15))
        gs = gridspec.GridSpec(2, 1, height_ratios=[3, 1])

        pyplot.clf()
        if self.prefix is None:
            #pyplot.suplots(nrows=1, ncols=2)
            pyplot.subplot(gs[0])
        x = y = numpy.linspace(-RANGE, RANGE, N_POINTS)
        pyplot.contour(x,y,disc_map.reshape((len(x), len(y))).T)

        true_dist = true_dist.cpu().numpy()
        perturbed = perturbed.cpu().numpy()
        pyplot.scatter(true_dist[:, 0], true_dist[:, 1], c='orange',marker='+')
        pyplot.scatter(perturbed[:, 0], perturbed[:, 1], c='red', marker='+')
        pyplot.scatter(samples[:, 0],   samples[:, 1],   c='green', marker='*')
        if self.prefix is not None:
          pyplot.savefig(self.prefix+'{:05d}'.format(self.frame_index)+'.jpg')
        else:
          pyplot.subplot(gs[1])
          pyplot.plot(losses)
          IPython.display.clear_output(wait=True)
          IPython.display.display(pyplot.gcf())
        self.frame_index += 1
    except Exception:
        print("some exception occurred while plotting")

In [9]:
class ToyGAN_G(nn.Module):
    def __init__(self, dim_hidden=512, dim_out=2, noise_dim=2):
        super(ToyGAN_G, self).__init__()
        self.dim_hidden, self.dim_out, self.noise_dim = dim_hidden, dim_out, noise_dim
        self.net = nn.Sequential(
            nn.Linear(noise_dim, dim_hidden),
            nn.LeakyReLU(),
            nn.Linear(dim_hidden, dim_hidden),
            nn.LeakyReLU(),
            nn.Linear(dim_hidden, dim_hidden),
            nn.LeakyReLU(),
            nn.Linear(dim_hidden, dim_out)
            )
    def forward(self, x):
        x = self.net(x)
        return x

class ToyGAN_D(nn.Module):
    def __init__(self, dim_hidden=512, dim_gen_out=2):
        super(ToyGAN_D, self).__init__()
        self.dim_hidden, self.dim_gen_out = dim_hidden, dim_gen_out
        self.net = nn.Sequential(
            nn.Linear(dim_gen_out, dim_hidden),
            nn.LeakyReLU(),
            nn.Linear(dim_hidden, dim_hidden),
            nn.LeakyReLU(),
            nn.Linear(dim_hidden, dim_hidden),
            nn.LeakyReLU(),
            nn.Linear(dim_hidden, 1)
            )
    def forward(self, x): #?
        x = self.net(x)
        return x

In [15]:
class AccLogger(object):
    def __init__(self, follow=None):
        self.follow = follow       
        self.errD = []
        self.errG = []
        self.scoreD_real = []
        self.scoreD_fake = []
        self.lip_loss = []

    def log(self, errD=None, errG=None, scoreD_real=None, scoreD_fake=None, lip_loss=None, **kw):
        self.errD.append(errD)
        self.errG.append(errG)
        self.scoreD_real.append(scoreD_real)
        self.scoreD_fake.append(scoreD_fake)
        self.lip_loss.append(lip_loss)
        if self.follow is not None:
            self.follow.log(errD=errD, errG=errG, scoreD_real=scoreD_real, scoreD_fake=scoreD_fake, lip_loss=lip_loss, **kw)
        
    def get_acc(self):
        return self.errD, self.errG, self.scoreD_real, self.scoreD_fake, self.lip_loss
    
    
class ProgressLogger(object):
    def __init__(self, imggen, follow=None, iterval=50):
        self.follow = follow
        self.imggen = imggen
        self.iterval = 50
        self.losses = []
        
    def log(self, _iter=None, niter=None, errD=None, errG=None, scoreD_real=None, scoreD_fake=None, lip_loss=None, 
            real=None, grad_points=None, **kw):
        if (_iter+1) % self.iterval == 0:
          self.losses.append(errD)
          if grad_points is None:
            grad_points = real
          self.imggen(real, grad_points, losses)
          print("Method: {} with penalty weight {}, {}-sided penalty".format(name, penalty_weight, "one" if onesided else "two"))
          print('[%d/%d] Loss_D: %f Loss_G: %f Loss_D_real: %f Loss_D_fake %f Loss_lip %f'
            % (batches+1, niter,
            errD.data[0], errG.data[0], scoreD_real.data[0], scoreD_fake.data[0], lip_loss.data[0]))
        if self.follow is not None:
            self.follow.log(_iter=_iter, niter=niter, errD=errD, errG=errG, scoreD_real=scoreD_real, scoreD_fake=scoreD_fake, lip_loss=lip_loss, grad_points=grad_points, real=real, **kw)

In [ ]:
def run_experiment(lip_mode=opt_lip_mode,    # WGAN/DRAGAN/...
                   onesided = opt_onesided,
                   penalty_weight = opt_penalty_weight, 
                   perturb_both = False, 
                   perturb_symmetric = False,
                   perturb_scale=1.,
                   niter=opt_niter, 
                   data_pregenerate=False,
                   modeD="critic",
                   dataset="swissroll",
                   cuda=opt_cuda,
                   batsize=256):
    
    netG = ToyGAN_G()
    netD = ToyGAN_D()
    
    if opt_adam:
        optD = optim.Adam(netD.parameters(), lr=opt_lrD)
        optG = optim.Adam(netG.parameters(), lr=opt_lrG)
    else:
        optD = optim.RMSprop(netD.parameters(), lr=opt_lrD)
        optG = optim.RMSprop(netG.parameters(), lr=opt_lrG)
    
    plogger = ProgressLogger(ImageGenerator(netG, netD), iterval=2)
    alogger = AccLogger(follow=plogger)
    
    gantrainer = q.GANTrainer(mode=lip_mode, modeD=modeD, one_sided=onesided, penalty_weight=penalty_weight, 
                              perturb_both=perturb_both, perturb_symmetric=perturb_symmetric, perturb_scale=perturb_scale, 
                              clamp_weights_rng=(opt_clamp_lower, opt_clamp_upper), optimizerD=optD, optimizerG=optG, 
                              logger=alogger)
    
    datagen = inf_train_gen(dataset, batsize)
    
    gantrainer.train(netD, netG, niter=niter, niterD=10, batsizeG=batsize, data_gen=datagen, cuda=cuda)
    
    return alogger.get_acc()

In [17]:
def run_n_experiments(n=1, savedir="experiments/", savename=None, save=True,
                      lip_mode=opt_lip_mode,
                      onesided=opt_onesided,
                      penalty_weight=opt_penalty_weight,
                      perturb_both=False,
                      perturb_symmetric=False,
                      niter=opt_niter,
                      data_pregenerate=False, 
                      modeD="critic", 
                      **kw):
    names = "WGAN WGAN-GP DRAGAN DRAGAN-G DRAGAN-LG".split()
    
    mat = numpy.zeros((niter, n, 5))
    cols = ["errD", "errG", "scoreD_real", "scoreD_fake", "lip_loss"]
    for ni in range(n):
        errD, errG, scoreD_real, scoreD_fake, lip_loss = run_experiment(lip_mode=lip_mode, 
                                                                        onesided=onesided, 
                                                                        penalty_weight=penalty_weight, 
                                                                        perturb_both=perturb_both,
                                                                        perturb_symmetric=perturb_symmetric,
                                                                        niter=niter,
                                                                        data_pregenerate=data_pregenerate,
                                                                        modeD=modeD,
                                                                        **kw)
        mat[:, ni, 0] = errD
        mat[:, ni, 1] = errG
        mat[:, ni, 2] = scoreD_real
        mat[:, ni, 3] = scoreD_fake
        mat[:, ni, 4] = lip_loss
        
    multiindex = [reduce(lambda x, y: x + y, [[i+1]*mat.shape[1] for i in range(mat.shape[0])], []),
                  reduce(lambda x, y: x + y, [[j+1 for j in range(mat.shape[1])] for i in range(mat.shape[0])], [])]
    
    flatmat = mat.reshape((-1, mat.shape[-1]))
    #flatmat = np.concatenate([np.asarray(multiindex).T, flatmat], axis=1)
    mi = pd.MultiIndex.from_arrays(multiindex, names="iter run".split())
    df = pd.DataFrame(flatmat, index=mi, columns=cols)
    
    if savename is None:
        extraoptstring = ""
        if lip_mode in "DRAGAN DRAGAN-G DRAGAN-LG":
            extraoptstring += "_pboth={}".format(int(perturb_both))
        if lip_mode == "DRAGAN":
            extraoptstring += "_psym={}".format(int(perturb_symmetric))
        savename = "{}_os={}_pw={}{}_niter={}_at_{}".format(
            lip_mode, int(onesided), penalty_weight, extraoptstring, niter,
            str(dt.now()).replace(" ", "_"))
    savep = savedir + savename
    if save:
        df.to_csv(savep)
        loadeddf = pd.read_csv(savep, index_col=[0, 1], skipinitialspace=True)
        #return loadeddf
    return df

In [18]:
def run_list_of_experiments(*specs):  # specs must be dictionaries (see example in next cell)
    for spec in specs:
        # rename long kw names
        rename_dic = {"lm": "lip_mode", 
                      "pw": "penalty_weight", 
                      "os": "onesided", 
                      "pboth": "perturb_both", 
                      "psym": "perturb_symmetric", 
                      "pscale": "perturb_scale"}
        for rde in rename_dic:
            if rde in spec:
                spec[rename_dic[rde]] = spec[rde]
                del spec[rde]
        run_n_experiments(**spec)

In [ ]:
run_experiment(lip_mode="WGAN-GP", cuda=False)